## Assignment
* Load and preprocess CIFAR100 dataset (not CIFAR10)
* Build a feedforward network for it. You can experiment around with number of layers and and neurons in each layer and different activation functions
* You are allowed to use nn.functional. (convolutions _might_ make your accuracy better)

In [ ]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
#using the GPU if it exists
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
#manual seed for reproducibility
torch.manual_seed(1234567890)

In [ ]:
#loading and preprocessing the data
#i am including data augmentation and also using common standard values for mean and std
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #adds a padding (4 px width) of zeros and crops to 32 * 32 randomly
    transforms.RandomHorizontalFlip(p=0.5), #flips the image horizontally with probability 0.5
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])

transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

train_dataset = datasets.CIFAR100(root="./data", train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR100(root="./data", train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)

In [ ]:
#to know the length of the datasets
print(f"training data: {len(train_dataset)}")
print(f"testing data: {len(test_dataset)}")
print(f"no of batches in training data: {len(train_loader)}")
print(f"no of batched in testing data: {len(test_loader)}")

training data: 50000
testing data: 10000
no of batches in training data: 391
no of batched in testing data: 10


In [ ]:
#Building the Model
SequentialNet = nn.Sequential(

    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0), #input shape: (128, 3, 32, 32), output shape: (128, 32, 30, 30)
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2), #input shape: (128, 32, 30, 30), #output shape: (128, 32, 15, 15)

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, padding=2), #input shape: (128, 32, 15, 15) #output shape: (128, 64, 16, 16)
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2), #input shape: (128, 64, 16, 16), output shape: (128, 64, 8, 8)

    nn.Flatten(), #output shape: (128, 4096)
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Linear(512, 100),

)

In [ ]:
#training loop
def train(model, loader, optimizer, loss_fn, epochs):
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    for x, y in loader:
      x, y = x.to(device), y.to(device)
      optimizer.zero_grad()
      logits = model(x)
      loss = loss_fn(logits, y)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss:.4f}")

#testing loop
def test(model, loader):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for x, y in loader:
      x, y = x.to(device), y.to(device)
      logits = model(x)
      pred = logits.argmax(dim=1)
      correct += (pred == y).sum().item()
      total += y.size(0)
  print(f"Accuracy: {100 * correct / total:.2f}%")


In [ ]:
#training
print("\nTraining SequentialNet")
sequential_model = SequentialNet.to(device)
optimizer_seq = optim.Adam(sequential_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


train(sequential_model, train_loader, optimizer_seq, loss_fn, epochs=20)
test(sequential_model, test_loader)


Training SequentialNet
Epoch: 0, Loss: 652.5024
Epoch: 1, Loss: 643.7968
Epoch: 2, Loss: 632.5286
Epoch: 3, Loss: 624.2970
Epoch: 4, Loss: 618.2031
Epoch: 5, Loss: 612.4785
Epoch: 6, Loss: 603.8041
Epoch: 7, Loss: 598.4736
Epoch: 8, Loss: 593.7300
Epoch: 9, Loss: 587.3079
Epoch: 10, Loss: 585.8922
Epoch: 11, Loss: 576.4255
Epoch: 12, Loss: 569.0679
Epoch: 13, Loss: 567.1825
Epoch: 14, Loss: 558.6471
Epoch: 15, Loss: 555.4223
Epoch: 16, Loss: 553.4626
Epoch: 17, Loss: 552.2862
Epoch: 18, Loss: 543.1860
Epoch: 19, Loss: 541.3658
Accuracy: 50.27%
